In [1]:
#import neccessary libraries
import pandas as pd
import numpy as np
import sklearn
import csv
import re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy

## Load main data

In [2]:
data = pd.read_csv('Data/newdata.csv',index_col=0)

In [3]:
data.head()

,date,sender,recipient1,recipient2,recipient3,subject,text
0,2001-05-14 23:39:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,NaN,NaN,"['', 'Here is our forecast', '', ' ']"
1,2001-05-04 20:51:00,phillip.allen@enron.com,john.lavorato@enron.com,NaN,NaN,Re:,"['', 'Traveling to have a business meeting tak..."
2,2000-10-18 10:00:00,phillip.allen@enron.com,leah.arsdall@enron.com,NaN,NaN,Re: test,"['', 'test successful. way to go!!!']"
3,2000-10-23 13:13:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,NaN,NaN,"['', 'Randy,', '', ' Can you send me a schedul..."
4,2000-08-31 12:07:00,phillip.allen@enron.com,greg.piper@enron.com,NaN,NaN,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]"


## Create priority keyword list

In [4]:
STOP = set(stopwords.words('english'))
words = pd.read_csv('./Data/U_Keyword_Dataset.txt', sep='/n', quoting=csv.QUOTE_NONE,
                    names=["Urgency_Keywords"])  # csv comma seperated value
words = words['Urgency_Keywords'].values  # to numpy
words = words.tolist()  # to list
print(words)

['limited', 'time', 'only', 'urgent', 'hurry', 'going', 'now', 'rush', 'last', 'chance', 'deadline', 'final', 'close', 'out', 'going', 'bussiness', 'day', 'never', 'again', 'clerence', "don't", 'deley', 'not', 'miss', 'expires', 'compelling', 'critical', 'crucial', 'demanding', 'essential', 'immediate', 'imperative', 'important', 'indispensable', 'necessary', 'persuasive', 'pressing', 'serious', 'vital', 'weighty', 'burning', 'called-for', 'capital', 'chief', 'clamant', 'clamorous', 'crying', 'demanded', 'driving', 'exigent', 'foremost', 'heavy', 'hurry-up', 'impelling', 'importunate', 'insistent', 'instant', 'leading', 'death', 'momentous', 'paramount', 'primary', 'principal', 'required', 'salient', 'top-priority', 'touch', 'touchy', 'quick', 'fast', 'wanted', 'ASAP', 'soon', 'possible']


/home/ratan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
ps =PorterStemmer()
SplitIntoWords = lambda message: TextBlob(message.lower()).words
WordsIntoBaseForm = lambda message: [ps.stem(word) for word in SplitIntoWords(message)]


sub_str = "Its urgently work please reply quick"
sub_str= WordsIntoBaseForm(sub_str)
print(sub_str)

['it', 'urgent', 'work', 'pleas', 'repli', 'quick']


## Preprocess Data

In [6]:
# Create an priorty for each row
data.text = data.text.apply(WordsIntoBaseForm)
res= []
for text in data.text:
    count = 0
    for sub_sub_str in text:
        if sub_sub_str in words:
            count +=1
    res.append(count)

In [7]:
# Create new column for priority
data['priority'] = res

## Sort by priority

In [8]:
data.sort_values(by=['priority','date'], ascending=False).head(10)

,date,sender,recipient1,recipient2,recipient3,subject,text,priority
1369,2001-12-21 14:10:07,showtimes@amazon.com,pallen@enron.com,NaN,NaN,Your Weekly Movie Showtimes from Amazon.com,"[amazon.com, weekli, movi, showtim, imag, imag...",41
683,2000-12-13 16:11:00,yild@zdemail.zdlists.com,pallen@enron.com,NaN,NaN,Y-Life Daily: Bush will almost definitely be p...,"['y-lif, daili, bulletin, decemb, 13, 2000, no...",22
1575,2000-12-13 16:11:00,yild@zdemail.zdlists.com,pallen@enron.com,NaN,NaN,Y-Life Daily: Bush will almost definitely be p...,"['y-lif, daili, bulletin, decemb, 13, 2000, no...",22
583,2000-10-24 13:29:00,phillip.allen@enron.com,keith.holst@enron.com,NaN,NaN,Investment Structure,"[forward, by, phillip, k, allen/hou/ect, on, 1...",21
1177,2000-10-24 13:29:00,phillip.allen@enron.com,keith.holst@enron.com,NaN,NaN,Investment Structure,"[forward, by, phillip, k, allen/hou/ect, on, 1...",21
1526,2000-10-24 13:29:00,phillip.allen@enron.com,keith.holst@enron.com,NaN,NaN,Investment Structure,"[forward, by, phillip, k, allen/hou/ect, on, 1...",21
25,2000-09-26 16:28:00,phillip.allen@enron.com,kholst@enron.com,NaN,NaN,Investment Structure,"[forward, by, phillip, k, allen/hou/ect, on, 0...",21
603,2000-09-26 16:28:00,phillip.allen@enron.com,kholst@enron.com,NaN,NaN,Investment Structure,"[forward, by, phillip, k, allen/hou/ect, on, 0...",21
1510,2000-09-26 16:28:00,phillip.allen@enron.com,kholst@enron.com,NaN,NaN,Investment Structure,"[forward, by, phillip, k, allen/hou/ect, on, 0...",21
26,2000-09-26 16:26:00,phillip.allen@enron.com,pallen70@hotmail.com,NaN,NaN,Investment Structure,"[forward, by, phillip, k, allen/hou/ect, on, 0...",21
